In [66]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine


In [67]:
# Paths
covid_crime_data = pd.read_csv('../Output/clean_covid_crime_data.csv')
income_data = pd.read_csv('../Output/clean_income_data.csv')
employment_data = pd.read_csv('../Output/employement_clean.csv')


In [68]:
# Postgres connection info
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5433'
database = 'covid_crime_db'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

In [69]:
# Connect to default database
default_engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/postgres')

In [94]:
# import contacts to sql
covid_crime_data.to_sql("crime", con=engine.connect(), if_exists='replace')
income_data.to_sql("income", con=engine.connect(), if_exists='replace')
employment_data.to_sql("employment", con=engine.connect(), if_exists='replace')

840

In [70]:
# Create a new database
try:
    # Connect to the database
    conn = default_engine.connect()
    
    # Create new database
    conn.execute("commit")
    conn.execute("CREATE DATABASE covid_crime_db")
    
    # Close the connection
    conn.close()
except OperationalError as e:
    print(f"Error occurred: {e}")

# Now connect to your newly created database
new_db_engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/covid_crime_db')

ProgrammingError: (psycopg2.errors.DuplicateDatabase) database "covid_crime_db" already exists

[SQL: CREATE DATABASE covid_crime_db]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [92]:
# Execute the SQL script to create the tables
try:
    with open(r'C:\Users\neilg\ut\homework\Group Project 3\Project3-group4\jupyter_notebooks\schema3.sql', 'r', encoding='utf-8-sig') as file:
        schema_sql = file.read()

    with engine.connect() as connection:
        trans = connection.begin()
        connection.execute(schema_sql)
        trans.commit()
    print("Tables created successfully.")
    
except SQLAlchemyError as e:
    print(f"An error occurred when creating tables: {e}")

An error occurred when creating tables: (psycopg2.errors.DuplicateTable) relation "crime_data_table" already exists

[SQL: CREATE TABLE "crime_data_table" (
    "Vector ID" varchar(255)   NOT NULL,
    "Year" date   NOT NULL,
    "Province" varchar   NOT NULL,
    "Crime Category" varchar   NOT NULL,
    "Crime Type" varchar   NOT NULL,
    "Coordinate" varchar   NOT NULL,
    "Value" int   NOT NULL    
);

CREATE TABLE "income_data_table" (
    "Vector ID" varchar(255)   NOT NULL,
    "Year" date   NOT NULL,
    "Province" varchar   NOT NULL,
    "Unit of Measure" varchar   NOT NULL,
    "Value" float   NOT NULL   
);

CREATE TABLE "employment_data_table" (
    "Vector ID" varchar(255)   NOT NULL,
    "Year" date   NOT NULL,
    "Labour force characteristics" varchar   NOT NULL,
    "sex" varchar   NOT NULL,
    "Unit of Measure" varchar   NOT NULL,
    "Value" float   NOT NULL,
    "Coordinate" varchar   NOT NULL    
);

]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [87]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import mapper, sessionmaker

# Define ORM classes
class CrimeData(object):
    pass

class IncomeData(object):
    pass

class EmploymentData(object):
    pass




In [88]:
# Create an engine (replace with your database URL)
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/covid_crime_db')



In [89]:
# Reflect the existing database tables
metadata = MetaData(engine)
metadata.reflect()

In [93]:
# Map the classes to the tables
mapper(CrimeData, metadata.tables['crime_data_table'])


ArgumentError: Class '<class '__main__.CrimeData'>' already has a primary mapper defined. 

In [78]:
mapper(IncomeData, metadata.tables['income_data_table'])

<Mapper at 0x26192d7fe50; IncomeData>

In [79]:
mapper(EmploymentData, metadata.tables['employment_data_table'])

<Mapper at 0x2618edb0190; EmploymentData>

In [80]:
# Create a session
Session = sessionmaker(bind=engine)
session = Session()

In [91]:
# Insert data from DataFrames
for index, row in covid_crime_data.iterrows():
    record = CrimeData(**row.to_dict())
    session.add(record)

TypeError: __init__() got an unexpected keyword argument 'Vector ID'

In [ ]:
# Commit the session to save changes
session.commit()

In [ ]:
# Close the session
session.close()